# 15.3 Elements of dynamic programming
There are two elements of DP: ***optimal substructure*** and ***overlapping subproblems***.

## 1. Optimal substructure
**Optimal substructure** demands the **independency of the subproblems**; you should be careful not to assume that optimal substructure applies when it does not. A counter example is the **Unweighted longest simple path**.

## 2. Overlapping subproblems
When a recursive algorithm revisits the same problem repeatedly, we say that the optimization problem has ***overlapping subproblems***. DP hence 1) solve each subproblem once and 2) store the solution in a table where it can be looked up when needed. This approach is called **recursion with memiozation**.

Here we show the:
1. `recursive_matrix_chain(p,i,j)`: A naive recursion of matrix-chain multiplication, which takes $\Theta (2^n)$ time
    
2. `memoized_matrix_chain(p,i,j)`: A memoized top-down version of the recusion, which takes $\Theta (n^3)$ time 

In [20]:
import numpy as np
p=np.array([30,35,15,5,10,20,25])

In [21]:
# naive recursion
def recursive_matrix_chain(p,i,j):
    if i==j:
        return 0
    # n=p.length-1
    n=len(p)-1
    #let m[1...n,1...n] to store the m[i,j] cost
    m=np.inf
    for k in range(i,j):
        q=recursive_matrix_chain(p,i,k)+recursive_matrix_chain(p,k+1,j)+p[i-1]*p[k]*p[j]
        if q<m:
            m=q
    return m
recursive_matrix_chain(p,1,6)    

15125

`recursive_matrix_chain2` uses `min()` function to derive $m$:

In [23]:
# naive recursion, alternative expression with min()
def recursive_matrix_chain2(p,i,j):
    if i==j:
        return 0
    # n=p.length-1
    n=len(p)-1
    #let m[1...n,1...n] to store the m[i,j] cost
    m=np.inf
    for k in range(i,j):
        m=min(m,recursive_matrix_chain(p,i,k)+recursive_matrix_chain(p,k+1,j)+p[i-1]*p[k]*p[j])
    return m
recursive_matrix_chain2(p,1,6)    

15125

`memoized_matrix_chain(p,i,j)` was slightly modified from the pseudocodes `memoized_matrix_chain(p)`on *p*388 of the textbook, so that:
* It intakes any $(i,j)$ with $1\leq i\leq j\leq n$
* It outputs $m[i,j]$, the minimum scalar multiplications for chain $A_i..A_j$ within chain $A_1..A_n$

In [19]:
def memoized_matrix_chain(p,a,b):
    n=len(p)-1 # n=p.length-1
    m=np.zeros((n+1,n+1)) #let m[1...n,1...n] to store the m[i,j] cost
    for i in range(a,b+1): #set the initivial value of any m[i,j]=inf as sentinel, where 1<=i<=j<=n
        for j in range(i,b+1):
            m[i,j]=np.inf
    return lookup_chain(m,p,a,b)
   
def lookup_chain(m,p,i,j):
    if m[i,j]<np.inf: #if m[i,j] was calculated before, look it up
        return m[i,j] #and return m[i,j]
    if i==j: #base case of recursion
        m[i,j]=0 #fill m[i,j] with 0
    else:
        for k in range(i,j): #recurrence
            q=lookup_chain(m,p,i,k)+lookup_chain(m,p,k+1,j)+p[i-1]*p[k]*p[j]
            if q<m[i,j]:
                m[i,j]=q #fill m[i,j] with the optimal cost q
    return m[i,j] #finally, return searched value
    
memoized_matrix_chain(p,1,6)    

15125.0

### Analysis of recursion with memoization
We can catagorize the calls of `lookup_chain` into two types:
1. Calls in which $m[i,j]=\infty$, so that Line 12-19 execute
2. calls in which $m[i,j]<\infty$, so that it simply returns $m[i,j]$ in Line 11

Because we calculate each entry in $m$ exactly once, there are $\Theta n^2$ calls of the first type. All calls of the second type are made as **recursive calls** by calls of the first type. Whenever a given call of `lookup_chain` makes recursive call, it makes $O(n)$ of them (i.e. from n till 1 at the base case). Therefore, there are $O(n^3)$ calls of the second type in all. Each call of the second type takes $O(1)$ time, and each call of the first type takes $O(n)$ time plus the time spent in its recursive calls. The total time is therefore $O(n^3)$.

### Comparison between bottom-up DP and recursion with memoization

**In theory**:
* Both methods take advantage of the **overlapping-subprobelms property**, so that all subproblems are solved only once
* They have the same running time

**In practice**:
* If all subproblems must be solved at least one, a bottom-up DP usually outperforms the corresponding recusion with memoization by a constant factor, because the bottom-up DP has
    * 1) no overhead for recursion and 
    * 2)less overhead for maintainig the table
* For some problems we can exploit the regular pattern of table accesses in bottom-up DP to reduce time or space requirements event further
* Alternatively, if some subproblems in the subproblem space need not be solved at all, the memoized solution has the advantage of solving only those subproblems that are definitely required